In [1]:
import keras
from keras import models, layers, optimizers
from keras.datasets import mnist, cifar10
import sys
sys.path.append('C:/Users/320060820/experiments/')
import os
from model_gen import keras_cnn
import copy

import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# to restart layer numbering
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [2]:
import scipy.io as sio

In [3]:
train_data = sio.loadmat('../datasets/svhn/train_32x32.mat')
test_data = sio.loadmat('../datasets/svhn/test_32x32.mat')

In [4]:
(x_train, y_train), (x_test, y_test) = (train_data['X'], train_data['y']), (test_data['X'], test_data['y'])

In [5]:
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

x_train = x_train[np.newaxis,...]
x_train = np.swapaxes(x_train,0,4).squeeze()

x_test = x_test[np.newaxis,...]
x_test = np.swapaxes(x_test,0,4).squeeze()

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [7]:
K.clear_session()
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))

opt = optimizers.rmsprop(lr=0.0001)

model.compile(loss='categorical_crossentropy', 
                optimizer = opt,
                metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=32,
                    shuffle=True, 
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 73257 samples, validate on 26032 samples
Epoch 1/50
73257/73257 [==============================] - 32s 439us/step - loss: 2.9962 - acc: 0.1463 - val_loss: 2.1709 - val_acc: 0.2497
Epoch 2/50
73257/73257 [==============================] - 31s 421us/step - loss: 2.3606 - acc: 0.2110 - val_loss: 1.9690 - val_acc: 0.3356
Epoch 3/50
73257/73257 [==============================] - 31s 417us/step - loss: 2.0843 - acc: 0.2872 - val_loss: 1.7709 - val_acc: 0.4461
Epoch 4/50
73257/73257 [==============================] - 31s 417us/step - loss: 1.8531 - acc: 0.3636 - val_loss: 1.5106 - val_acc: 0.5468
Epoch 5/50
73257/73257 [==============================] - 31s 418us/step - loss: 1.6517 - acc: 0.4408 - val_loss: 1.2656 - val_acc: 0.6623
Epoch 6/50
73257/73257 [==============================] - 30s 416us/step - loss: 1.4728 - acc: 0.5044 - val_loss: 1.1224 - val_ac

In [9]:
model.save('../test_models/svhn_keras/svhn_keras.h5')
model.save_weights('../test_models/svhn_keras/svhn_keras_w.h5')